#AI Group Project ~ Deep Learning

##Prediction of Pumpkin Seed Type



Dataset ini berisi 2.500 data dan 13 features. Terdapat dua jenis biji labu, yaitu 'Urgup_Sivrisi' dan 'Cercevelik' yang umumnya ditanam di daerah Urgup dan Karacaoren di Turki. Project ini dilakukan untuk dapat mengklasifikasi jenis biji labu dengan data-data yang telah ada. Dataset diambil dari link https://www.kaggle.com/datasets/muratkokludataset/pumpkin-seeds-dataset

In [22]:
#@title Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
import math
from collections import defaultdict
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import os
import seaborn as sns
import re
from numpy import random
from sklearn import preprocessing
import random
from random import seed
from random import randrange
from csv import reader
from math import sqrt
import csv
from tqdm import tqdm
from joblib import Parallel, delayed
!pip install kaggle #For importing from Kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
from google.colab import files #For kaggle key
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"sstarchange","key":"135de53606e44830e48536852b1d5dc7"}'}

In [24]:
!mkdir ~/.kaggle #Makes directory to store key
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [25]:
!kaggle datasets download -d muratkokludataset/pumpkin-seeds-dataset/code #Downloads dataset
!unzip pumpkin-seeds-dataset.zip #Unzips dataset

pumpkin-seeds-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  pumpkin-seeds-dataset.zip
replace Pumpkin_Seeds_Dataset/Pumpkin_Seeds_Dataset.arff? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [26]:
data = pd.read_excel("/content/Pumpkin_Seeds_Dataset/Pumpkin_Seeds_Dataset.xlsx") #Reads data
data.to_csv("Pumpkin_Seeds_Dataset.csv", index = False) #Index = False ensures that the row index is not included in the exported file
class_map = {'Çerçevelik': 0, 'Ürgüp Sivrisi': 1} #Changes class to simplify
data['Class'] = data['Class'].replace(class_map)
data

,Area,Perimeter,Major_Axis_Length,Minor_Axis_Length,Convex_Area,Equiv_Diameter,Eccentricity,Solidity,Extent,Roundness,Aspect_Ration,Compactness,Class
0,56276,888.242,326.1485,220.2388,56831,267.6805,0.7376,0.9902,0.7453,0.8963,1.4809,0.8207,0
1,76631,1068.146,417.1932,234.2289,77280,312.3614,0.8275,0.9916,0.7151,0.8440,1.7811,0.7487,0
2,71623,1082.987,435.8328,211.0457,72663,301.9822,0.8749,0.9857,0.7400,0.7674,2.0651,0.6929,0
3,66458,992.051,381.5638,222.5322,67118,290.8899,0.8123,0.9902,0.7396,0.8486,1.7146,0.7624,0
4,66107,998.146,383.8883,220.4545,67117,290.1207,0.8187,0.9850,0.6752,0.8338,1.7413,0.7557,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,79637,1224.710,533.1513,190.4367,80381,318.4289,0.9340,0.9907,0.4888,0.6672,2.7996,0.5973,1
2496,69647,1084.318,462.9416,191.8210,70216,297.7874,0.9101,0.9919,0.6002,0.7444,2.4134,0.6433,1
2497,87994,1210.314,507.2200,222.1872,88702,334.7199,0.8990,0.9920,0.7643,0.7549,2.2828,0.6599,1
2498,80011,1182.947,501.9065,204.7531,80902,319.1758,0.9130,0.9890,0.7374,0.7185,2.4513,0.6359,1


In [27]:
has_null = data.isnull().values.any() #Checks if there is null values

if has_null:
    print("Data contains null values")
else:
    print("Data does not contain null values")

Data does not contain null values


In [28]:
#Select specific columns from the DataFrame data and assigns the corresponding values to the variable X. 
#The '.values' attribute returns these column values as a NumPy array, which can be used for further computations or modeling tasks.
X = data[['Area',	'Perimeter',	'Major_Axis_Length',	'Minor_Axis_Length', 'Convex_Area',	'Equiv_Diameter',	'Eccentricity',	'Solidity',	'Extent',	'Roundness',	'Aspect_Ration',	'Compactness']].values
X

array([[5.627600e+04, 8.882420e+02, 3.261485e+02, ..., 8.963000e-01,
        1.480900e+00, 8.207000e-01],
       [7.663100e+04, 1.068146e+03, 4.171932e+02, ..., 8.440000e-01,
        1.781100e+00, 7.487000e-01],
       [7.162300e+04, 1.082987e+03, 4.358328e+02, ..., 7.674000e-01,
        2.065100e+00, 6.929000e-01],
       ...,
       [8.799400e+04, 1.210314e+03, 5.072200e+02, ..., 7.549000e-01,
        2.282800e+00, 6.599000e-01],
       [8.001100e+04, 1.182947e+03, 5.019065e+02, ..., 7.185000e-01,
        2.451300e+00, 6.359000e-01],
       [8.493400e+04, 1.159933e+03, 4.628951e+02, ..., 7.933000e-01,
        1.973500e+00, 7.104000e-01]])

In [29]:
#Select a specific column from the DataFrame data and assigns the corresponding values to the variable y. 
#The '.values' attribute returns these column values as a NumPy array, which can be used for further computations or modeling tasks.
y = data['Class'].values
y

array([0, 0, 0, ..., 1, 1, 1])

In [30]:
#Splits data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
  #@title XGB AI

# Class representing a single decision tree in the XGBoost ensemble
class XGBoostTree:
    def __init__(self, max_depth=3, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.left = None
        self.right = None
        self.feature_idx = None
        self.threshold = None
        self.value = None

    # Fit the tree to the data
    def fit(self, X, gradient, hessian, depth=0):
        # Check if stopping criteria are met
        if depth == self.max_depth or len(X) < self.min_samples_split:
            self.value = self.compute_gamma(gradient, hessian)
            return

        # Find the best feature and threshold to split the data
        best_gain = 0.0
        best_feature_idx = None
        best_threshold = None
        for feature_idx in range(X.shape[1]):
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                gain = self.compute_gain(X, gradient, hessian, feature_idx, threshold)
                if gain > best_gain:
                    best_gain = gain
                    best_feature_idx = feature_idx
                    best_threshold = threshold

        # If no gain is achieved, set the leaf node value
        if best_gain == 0.0:
            self.value = self.compute_gamma(gradient, hessian)
            return

        # Split the data based on the best feature and threshold
        self.feature_idx = best_feature_idx
        self.threshold = best_threshold

        # Recursively fit the left and right subtrees
        left_mask = X[:, best_feature_idx] <= best_threshold
        right_mask = X[:, best_feature_idx] > best_threshold

        self.left = XGBoostTree(self.max_depth, self.min_samples_split)
        self.left.fit(X[left_mask], gradient[left_mask], hessian[left_mask], depth + 1)

        self.right = XGBoostTree(self.max_depth, self.min_samples_split)
        self.right.fit(X[right_mask], gradient[right_mask], hessian[right_mask], depth + 1)

    def compute_gamma(self, gradient, hessian):
        return -np.sum(gradient) / (np.sum(hessian) + 1e-10)

    def compute_gain(self, X, gradient, hessian, feature_idx, threshold):
        left_mask = X[:, feature_idx] <= threshold
        right_mask = X[:, feature_idx] > threshold

        left_gradient = gradient[left_mask]
        left_hessian = hessian[left_mask]
        right_gradient = gradient[right_mask]
        right_hessian = hessian[right_mask]

        gain = 0.5 * ((np.sum(left_gradient) ** 2) / (np.sum(left_hessian) + 1e-10) +
                      (np.sum(right_gradient) ** 2) / (np.sum(right_hessian) + 1e-10) -
                      ((np.sum(left_gradient) + np.sum(right_gradient)) ** 2) /
                      (np.sum(left_hessian) + np.sum(right_hessian) + 1e-10))

        return gain

    def predict(self, X):
        if self.value is not None:
            return np.full(len(X), self.value)

        left_mask = X[:, self.feature_idx] <= self.threshold
        right_mask = X[:, self.feature_idx] > self.threshold

        y_pred = np.zeros(len(X))
        y_pred[left_mask] = self.left.predict(X[left_mask])
        y_pred[right_mask] = self.right.predict(X[right_mask])

        return y_pred


class XGBoostClassifier:
    def __init__(self, params):
        self.params = params
        self.n_estimators = params.get('n_estimators', 100)
        self.learning_rate = params.get('learning_rate', 0.1)
        self.max_depth = params.get('max_depth', 3)
        self.min_samples_split = params.get('min_samples_split', 2)
        self.estimators = []

    @staticmethod
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    def log_loss_gradient(self, y, y_pred):
        return self.sigmoid(y_pred) - y

    def log_loss_hessian(self, y, y_pred):
        preds = self.sigmoid(y_pred)
        return preds * (1 - preds)

    def fit(self, X, y):
        y_pred = np.zeros_like(y, dtype=float)
        for _ in range(self.n_estimators):
            tree = XGBoostTree(self.max_depth, self.min_samples_split)
            gradient = self.log_loss_gradient(y, y_pred)
            hessian = self.log_loss_hessian(y, y_pred)
            tree.fit(X, gradient, hessian)
            y_pred += self.learning_rate * tree.predict(X)
            self.estimators.append(tree)

    def predict_proba(self, X):
        y_pred = np.zeros(len(X))
        for estimator in self.estimators:
            y_pred += self.learning_rate * estimator.predict(X)

        proba = self.sigmoid(y_pred)
        return np.column_stack((1 - proba, proba))

    def predict(self, X):
        proba = self.predict_proba(X)
        return np.argmax(proba, axis=1)

    def kfold_cross_validation(self, X, y, k = 5):
        accuracy_scores = []
        kf = KFold(n_splits=k)

        progress_bar = tqdm(total=k, desc="K-Fold Progress")

        for train_index, val_index in kf.split(X):
            X_train, X_val = X[train_index], X[val_index]
            y_train, y_val = y[train_index], y[val_index]

            xgb_model = XGBoostClassifier(self.params)
            xgb_model.fit(X_train, y_train)
            y_pred = xgb_model.predict(X_val)
            accuracy = accuracy_score(y_val, y_pred)
            accuracy_scores.append(accuracy)

            progress_bar.update(1)
            progress_bar.set_postfix({"Accuracy": accuracy})

        progress_bar.close()

        average_accuracy = np.mean(accuracy_scores)
        return average_accuracy

In [35]:
#Set parameters for XGBoost
params = {
    'n_estimators': 10,
    'learning_rate': 0.1,
    'max_depth': 7,
    'min_samples_split': 2
}

In [ ]:
#Calls the XGBoostClassifier with set params. Also count the average accuracy from the K-Fold CV
xgb_model = XGBoostClassifier(params)
average_accuracy = xgb_model.kfold_cross_validation(X_train, y_train)

In [17]:
#@title n-Iterated K-Fold Cross Validation with Shuffled Datas

params = {
    'n_estimators': 10,
    'learning_rate': 0.1,
    'max_depth': 7,
    'min_samples_split': 2
}

X = X_train # Your input data
y = y_train  # Your target variable
k = 5  # Number of folds
n = 5   # Number of iterations

average_accuracies = []

progress_bar_outer = tqdm(total=n, desc="Iteration Progress")  # Outer progress bar

for _ in range(n):
    # Creates K-fold CV with random shuffling
    kf = KFold(n_splits=k, shuffle=True)

    accuracy_scores = []  # Accuracy scores for each fold

    progress_bar_inner = tqdm(total=k, desc="K-Fold Progress", leave=False)  # Inner progress bar

    for train_index, val_index in kf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        # Train and evaluate your model on each fold
        xgb_model = XGBoostClassifier(params)
        xgb_model.fit(X_train, y_train)
        y_pred = xgb_model.predict(X_val)

        # Compute the accuracy score for the fold
        accuracy = accuracy_score(y_val, y_pred)
        accuracy_scores.append(accuracy)

        progress_bar_inner.set_postfix({"Accuracy": accuracy})
        progress_bar_inner.update(1)

    progress_bar_inner.close()

    # Compute average accuracy across all folds for THIS iteration
    average_accuracy = np.mean(accuracy_scores)
    average_accuracies.append(average_accuracy)

    progress_bar_outer.set_postfix({"Average Accuracy": average_accuracy})
    progress_bar_outer.update(1)

progress_bar_outer.close()

# Compute average accuracy across ALL iterations
overall_average_accuracy = np.mean(average_accuracies)

# Print overall average accuracy
print("Overall Average Accuracy:", overall_average_accuracy)

Iteration Progress: 100%|██████████| 5/5 [20:20<00:00, 244.15s/it, Average Accuracy=0.883]

Overall Average Accuracy: 0.8794444444444445
